In [ ]:
# Tasks perform in this Notebook taken from various platforms like linkedin

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("GlobalSparkSession").getOrCreate()


24/02/10 19:45:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
Task - The dataset includes company_id, company_name, and multiple date columns representing the stock prices of the company. 
The task is to generate a new dataset with only three columns, "company_name", "Date", "Stock_price".
You have to unpivot the data

In [2]:
data = [
    (1, "Quantum Innovations", 12.5, 14.5, 16.5, 18.5, 20.5),
    (2, "Stellar Solutions", 14.5, 16.5, 18.5, 20.5, 22.5),
    (3, "Nebula Dynamics", 16.5, 18.5, 20.5, 22.5, 24.5),
    (4, "Fusion Enterprises", 18.5, 20.5, 22.5, 24.5, 26.5),
    (5, "Celestial Technologies", 20.5, 22.5, 24.5, 26.5, 28.5),
]
columns=["c_id","Company_Name","2024-02-01","2024-02-02","2024-02-03","2024-02-04","2024-02-05"]
df=spark.createDataFrame(data,columns)
df.show()

+----+--------------------+----------+----------+----------+----------+----------+
|c_id|        Company_Name|2024-02-01|2024-02-02|2024-02-03|2024-02-04|2024-02-05|
+----+--------------------+----------+----------+----------+----------+----------+
|   1| Quantum Innovations|      12.5|      14.5|      16.5|      18.5|      20.5|
|   2|   Stellar Solutions|      14.5|      16.5|      18.5|      20.5|      22.5|
|   3|     Nebula Dynamics|      16.5|      18.5|      20.5|      22.5|      24.5|
|   4|  Fusion Enterprises|      18.5|      20.5|      22.5|      24.5|      26.5|
|   5|Celestial Technol...|      20.5|      22.5|      24.5|      26.5|      28.5|
+----+--------------------+----------+----------+----------+----------+----------+



In [15]:
from pyspark.sql.functions import col,when
dates_col=["Dates"]
dates_data=[("2024-02-01",),("2024-02-02",),("2024-02-03",),("2024-02-04",),("2024-02-05",)]
dates_df=spark.createDataFrame(dates_data,dates_col)
resultset=df.crossJoin(dates_df)
final_result=resultset.withColumn("Stock_Price",
                      when(col("Dates")=='2024-02-01',col('2024-02-01'))
                     .when(col("Dates")=='2024-02-02',col('2024-02-02'))
                     .when(col("Dates")=='2024-02-03',col('2024-02-03'))
                     .when(col("Dates")=='2024-02-04',col('2024-02-04'))
                     .when(col("Dates")=='2024-02-05',col('2024-02-05'))
                      )
# use sparksql to select required columns
final_result.select("c_id","Company_Name","Dates","Stock_Price").show()
# dates_df.show()
              

+----+--------------------+----------+-----------+
|c_id|        Company_Name|     Dates|Stock_Price|
+----+--------------------+----------+-----------+
|   1| Quantum Innovations|2024-02-01|       12.5|
|   1| Quantum Innovations|2024-02-02|       14.5|
|   1| Quantum Innovations|2024-02-03|       16.5|
|   1| Quantum Innovations|2024-02-04|       18.5|
|   1| Quantum Innovations|2024-02-05|       20.5|
|   2|   Stellar Solutions|2024-02-01|       14.5|
|   2|   Stellar Solutions|2024-02-02|       16.5|
|   2|   Stellar Solutions|2024-02-03|       18.5|
|   2|   Stellar Solutions|2024-02-04|       20.5|
|   2|   Stellar Solutions|2024-02-05|       22.5|
|   3|     Nebula Dynamics|2024-02-01|       16.5|
|   3|     Nebula Dynamics|2024-02-02|       18.5|
|   3|     Nebula Dynamics|2024-02-03|       20.5|
|   3|     Nebula Dynamics|2024-02-04|       22.5|
|   3|     Nebula Dynamics|2024-02-05|       24.5|
|   4|  Fusion Enterprises|2024-02-01|       18.5|
|   5|Celestial Technol...|2024

In [43]:
# Do the above task using SparkSQL
# create temp sql table for values from dates_df dataframe
columns2=["c_id","Company_Name","feb1","feb2","feb3","feb4","feb5"]
df2=spark.createDataFrame(data,columns2)
temp_values=[("2024-02-01",),("2024-02-02",),("2024-02-03",),("2024-02-04",),("2024-02-05",)]
dates_df.createOrReplaceTempView("temp_values1")
df2.createOrReplaceTempView("temp_data1")
# df2.show()
results=spark.sql(f"select c_id,Company_Name,{'Dates'},case Dates when '{'2024-02-01'}' then feb1 when '{'2024-02-02'}' then feb2 when '{'2024-02-03'}' then feb3 when '{'2024-02-04'}' then feb4 else feb5 end as Stock_Price from temp_values1 cross join temp_data1")


In [1]:
# Write the results into parque file
# results.write.parquet('/home/BigData/Files/stock_price.parquet')
results.write.csv('/home/BigData/Files/stock_price.csv')

# results.show()

In [26]:
# Task :we have a dataset representing books issued by students, and we want to aggregate the books for each student.
# Multiple books should be shown separated by ";"
import pyspark.sql.functions as F

book_issued = [
    (101 ,'Mark' , "White Tiger"),
    (102 ,'Ria' , "The Fountainhead"),
    (102 ,'Ria' , "The Seceret History"),
    (101 ,'Mark' , "Bhagwad Gita"),
    (103 ,'Loi' , "The Fountainhead"),
    ]
books_schema=['s_id','s_name','book_names']
books_df=spark.createDataFrame(book_issued,books_schema)
gp=books_df.groupBy("s_id","s_name")\
.agg(F.concat_ws(';',F.collect_list("book_names")).alias("all_books"))\
.show(truncate=False)
# print(type(gp))

+----+------+------------------------------------+
|s_id|s_name|all_books                           |
+----+------+------------------------------------+
|101 |Mark  |White Tiger;Bhagwad Gita            |
|102 |Ria   |The Fountainhead;The Seceret History|
|103 |Loi   |The Fountainhead                    |
+----+------+------------------------------------+



In [3]:
'''
1.'Arranging a column with null values in ascending or descending order.'

Task - Sort the salary column of the dataframe :

'''
from pyspark.sql.types import *
import pyspark.sql.functions as F
data = [
    (1, "Aditya Sen" , None),
    (2, "Bikramaditya" , 200),
    (3, "Mark T" , 100),
    (4, "D K aditya" ,None),
    (5, "Danny" , 500),
    (6, "Eli" , 300),
    ]
cust_schema=StructType([
    StructField("id",IntegerType(),False), #is Nullable
    StructField("Name",StringType(),False),
    StructField("Amount",IntegerType(),True)
])
cust_df=spark.createDataFrame(data,cust_schema)
# print(cust_df)
# cust_df.show()
# a) in ascending order with NULL values at the top
cust_df.orderBy("Amount").show()
# b) in ascending order with NULL values at the bottom

# c) in descending order with NULL values at the top

# d) in descending order with NULL values at the bottom
cust_df.orderBy(F.desc("Amount")).show()


+---+------------+------+
| id|        Name|Amount|
+---+------------+------+
|  1|  Aditya Sen|  NULL|
|  4|  D K aditya|  NULL|
|  3|      Mark T|   100|
|  2|Bikramaditya|   200|
|  6|         Eli|   300|
|  5|       Danny|   500|
+---+------------+------+

+---+------------+------+
| id|        Name|Amount|
+---+------------+------+
|  5|       Danny|   500|
|  6|         Eli|   300|
|  2|Bikramaditya|   200|
|  3|      Mark T|   100|
|  1|  Aditya Sen|  NULL|
|  4|  D K aditya|  NULL|
+---+------------+------+



In [20]:
'''
Task - In your dataset, the "order_date" column includes a time component set to 00:00:00.
Your objective is to eliminate the time portion from the column, thereby conserving storage space.
'Remove time part from PySpark dataframe date column'
'''
from pyspark.sql.functions import col
data = [(1, "2024-02-01 00:00:00"),
        (2, "2024-02-01 00:00:00"),
        (3, "2024-02-03 00:00:00"),
        (4, "2024-02-04 00:00:00",)]
columns = ["Order_id","Order_date"]
df=spark.createDataFrame(data,columns)
# using sparksql
df.createOrReplaceTempView("orders")
# print(df)
removedTime = spark.sql("select Order_id,cast(substring(Order_date,1,10) as date) as order_date from orders")
# print(removedTime)

# using Pyspark
df.withColumn("Order_date2",col("Order_date")[0:10]).show()
removedTime.show()


+--------+-------------------+-----------+
|Order_id|         Order_date|Order_date2|
+--------+-------------------+-----------+
|       1|2024-02-01 00:00:00| 2024-02-01|
|       2|2024-02-01 00:00:00| 2024-02-01|
|       3|2024-02-03 00:00:00| 2024-02-03|
|       4|2024-02-04 00:00:00| 2024-02-04|
+--------+-------------------+-----------+

+--------+----------+
|Order_id|order_date|
+--------+----------+
|       1|2024-02-01|
|       2|2024-02-01|
|       3|2024-02-03|
|       4|2024-02-04|
+--------+----------+



In [14]:
'''case insensitive/ignorant data search'
Task - In the provided dataset, identify the rows where the 'Name' column contains the word 'aditya'.
'''
from pyspark.sql.functions import filter

data = [
    (1, "Aditya Sen" , 100),
    (2, "Bikramaditya" , 200),
    (3, "Mark T" , 100),
    (4, "D K aditya" , 200),
    (5, "Danny" , 500),
    (6, "Eli" , 100),
]
columns = ["id","name","Amount"]
df=spark.createDataFrame(data,columns)
df.createOrReplaceTempView("users")
# spark.sql("select id,name,Amount from users where lower(name) like '%aditya%'").show()
# df.where("lower(name) like '%aditya%'").show()
# df.filter(df["name"]).rlike('aditya').show()

In [28]:
# Task - Show work group wise , average salary for each department
# Suggestion: instead of following sql like approach follow, spark approach

from pyspark.sql.types import ArrayType
from pyspark.sql.types import *
from pyspark.sql.functions import col,when,avg
import pandas as pd
spark = spark.builder.getOrCreate()

schema=StructType([
    StructField("FirstName",ArrayType(StringType()),True),
    StructField("LastName",ArrayType(StringType()),True),
    StructField("Type",ArrayType(StringType()),True),
    StructField("Dept",ArrayType(StringType()),True),
    StructField("Yoe",ArrayType(IntegerType()),True),
    StructField("Sal",ArrayType(IntegerType()),True)
])
data = {'First Name': ['Aryan', 'Rohan', 'Riya', 'Yash', 'Siddhant'],
        'Last Name': ['Singh', 'Agarwal', 'Shah', 'Bhatia', 'Khanna'],
        'Type': ['Full-time Employee', 'Intern', 'Full-time Employee', 'Part-time Employee', 'Full-time Employee'],
        'Department': ['Administration', 'Technical', 'Administration', 'Technical', 'Management'],
        'YoE': [2, 3, 5, 7, 6],
        'Salary': [20000, 5000, 10000, 10000, 20000]
       }

pandas_df=pd.DataFrame(data)
# print(pandas_df)
# data1=(list(data.values()), list(data.keys()))
# df = spark.createDataFrame(list(data.values()), list(data.keys()))

df=spark.createDataFrame(pandas_df);
grouped_data=df.groupBy("Type","Department").agg(avg('Salary').alias("Avg_Salary"))#,min("Department").alias("Department"))
grouped_data.show()
'''
gd=grouped_data.withColumn("Administration",when(col("Department")=='Administration',col('Avg_Salary')).otherwise(0))\
            .withColumn("Technical",when(col("Department")=='Technical',col('Avg_Salary')).otherwise(0))\
            .withColumn("Management",when(col("Department")=='Management',col('Avg_Salary')).otherwise(0))
# gd.select("Type","Administration","Management","Technical").show()
'''
df.groupBy("Type").pivot("Department").avg("Salary").na.fill(0).alias("Avg_Sal").show()

# gd.show()
# print(data1)

+------------------+--------------+----------+
|              Type|    Department|Avg_Salary|
+------------------+--------------+----------+
|Full-time Employee|Administration|   15000.0|
|            Intern|     Technical|    5000.0|
|Full-time Employee|    Management|   20000.0|
|Part-time Employee|     Technical|   10000.0|
+------------------+--------------+----------+

+------------------+--------------+----------+---------+
|              Type|Administration|Management|Technical|
+------------------+--------------+----------+---------+
|Full-time Employee|       15000.0|   20000.0|      0.0|
|            Intern|           0.0|       0.0|   5000.0|
|Part-time Employee|           0.0|       0.0|  10000.0|
+------------------+--------------+----------+---------+



In [11]:
'''
Task - You are provided with two datasets, branch1 and branch2 , representing information about students and
their marks in different subjects across different branches. Your goal is to combine these datasets into one final dataset.
Missing text information should be shown as 'unknown' , and missing numerical information should be shown as -9999.
'''
from pyspark.sql.functions import lit
branch1 = spark.createDataFrame([["Delhi", "Neha", 90]], ["Branch", "Student", "Maths_marks"])
branch2 = spark.createDataFrame([["Arav","Kolkata", 79, 83], [None,"Kolkata", 89, 73]],["Student", "Branch", "Science_marks", "Maths_marks"])

branch_v1=branch1.withColumn("Science_marks",lit(-9999)).select("Student","Branch","Science_marks","Maths_marks")
union_data=branch_v1.union(branch2).selectExpr("Branch","case when Student is null then 'Unknown' else Student end as Student","Maths_marks","Science_marks")
union_data.show()

+-------+-------+-----------+-------------+
| Branch|Student|Maths_marks|Science_marks|
+-------+-------+-----------+-------------+
|  Delhi|   Neha|         90|        -9999|
|Kolkata|   Arav|         83|           79|
|Kolkata|Unknown|         73|           89|
+-------+-------+-----------+-------------+



In [1]:
'''
Task - The dataset contains information about food items in bills.
Your assignment is to determine the frequency of each food item ordered.
'''
from pyspark.sql.types import StructType,StructField,IntegerType,ArrayType,StringType
from pyspark.sql.functions import explode,count
import pandas as pd
data = [
(101, ["dosa", "biriyani", "idli"]),
(102, ["biriyani", "mineral water"]),
(103, ["rice", "mineral water", "poha"]),
(109, ["idli", "biriyani", "poha"]),
]

schema=StructType([
    StructField("id",IntegerType(),True),
    StructField("orders",ArrayType(StringType()),True)
])

df=spark.createDataFrame(data,schema).select(explode("orders").alias("orders"))
getCount=df.groupBy("orders").count()
getCount.orderBy(["count"],ascending=False)#.show(truncate=False)
# exploded=df.withColumn("items",explode(df.orders))
# exploded.show(truncate=False)
# orders.printSchema()

DataFrame[orders: string, count: bigint]

In [6]:
# Optimized code for the above

df=spark.createDataFrame(data,schema).select(explode("orders").alias("orders")) \
.groupBy("orders") \
.count().orderBy("count",ascending=False) \
.show()

+-------------+-----+
|       orders|count|
+-------------+-----+
|     biriyani|    3|
|         idli|    2|
|mineral water|    2|
|         poha|    2|
|         dosa|    1|
|         rice|    1|
+-------------+-----+



In [8]:
'''
Task - Transform the input DataFrame into a new DataFrame , where each row represents a unique student
and the columns include the student's name along with the marks for the "math" and "eng" subjects.
'''
from pyspark.sql.types import StructType,StructField,IntegerType,ArrayType,StringType

data=[
('Rudra','math',79),
('Rudra','eng',60),
('Shivu','math', 68),
('Shivu','eng', 59),
('Anu','math', 65),
('Anu','eng',80)
]
schema=StructType([
    StructField("Name",StringType(),True),
    StructField("Sub",StringType(),True),
    StructField("Marks",IntegerType(),True)
])
df=spark.createDataFrame(data,schema)\
.show()


+-----+----+-----+
| Name| Sub|Marks|
+-----+----+-----+
|Rudra|math|   79|
|Rudra| eng|   60|
|Shivu|math|   68|
|Shivu| eng|   59|
|  Anu|math|   65|
|  Anu| eng|   80|
+-----+----+-----+



In [ ]:
# Task: Read a table data from the sources like SQL Server/Hive/datalake


In [40]:
"""
Question: Given a list of sales transactions with product names and their corresponding sales amounts,
calculate the total sales for each product and store the results in a dictionary.
"""
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

spark=SparkSession.builder.appName("interview_prep").getOrCreate()
sales = [("Widget", 500), ("Gadget", 750), ("Widget", 1000), ("Doodad", 1200), ("Gadget", 600)]
sales_df=spark.createDataFrame(sales,schema=["product","price"])
sales_df=sales_df.groupBy("product").agg(sum("price").alias("sum_sal"))
sales_df.show()
newdf=sales_df.select("product","sum_sal")
newdf.show()
#.\
# product_sales_dict = sales_df.to_dict()
# print(sales_df)

+-------+-------+
|product|sum_sal|
+-------+-------+
| Widget|   1500|
| Gadget|   1350|
| Doodad|   1200|
+-------+-------+

+-------+-------+
|product|sum_sal|
+-------+-------+
| Widget|   1500|
| Gadget|   1350|
| Doodad|   1200|
+-------+-------+



In [70]:
# above problem using RDD
sales = [("Widget", 500), ("Gadget", 750), ("Widget", 1000), ("Doodad", 1200), ("Gadget", 600)]
rdd=sc.parallelize(sales)

# you have to give column index because RDD dont have column names.
# any transformation returns pipelined RDD
groupData=rdd.reduceByKey(lambda x,y:x+y).collectAsMap()
# data=groupData.collect()
print(groupData)

# print(type(data))
# print(groupData)
# unpacked_data = {key: list(value) for key, value in groupData.items()}
# print(unpacked_data)

{'Doodad': 1200, 'Widget': 1500, 'Gadget': 1350}
<class 'list'>


In [ ]:
# ggg